## Loading data

In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-11-03 01:07:21--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.98.207, 74.125.134.207, 142.251.107.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.98.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  79.2MB/s    in 0.8s    

2024-11-03 01:07:22 (79.2 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Reading the Data into arrays

In [4]:
from PIL import Image
import numpy as np

NUMBER_OF_EXAMPLES = 10

x_train = []
y_train = []

cats_dir = os.path.join(train_dir + "/cats")
dogs_dir = os.path.join(train_dir + "/dogs")



i = 0
while i < NUMBER_OF_EXAMPLES:
  if i % 2 == 0:
    im = Image.open(os.path.join(cats_dir, os.listdir(cats_dir)[i])).convert("RGB")

    #im_resized is done so that they are all of the same pixel size, they are shrunk to the same size.
    im_resized = im.resize((150,150))

    x_train.append(np.array(im_resized))
    y_train.append(1)
  else:
    im = Image.open(os.path.join(dogs_dir, os.listdir(dogs_dir)[i])).convert("RGB")

    #im_resized is done so that they are all of the same pixel size, they are shrunk to the same size.
    im_resized = im.resize((150,150))

    x_train.append(np.array(im_resized))
    y_train.append(0)
  i += 1

## Beginning to define the model (this is where you come in, I loaded the pretrained model for you)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [6]:
import tensorflow as tf

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [7]:
# from tensorflow.keras.layers import Resizing


# # Add a resizing layer at the beginning of your model
# resnet_model = Sequential([
#    Resizing(180, 180, interpolation='bilinear', input_shape=(150, 150, 3)),
#    pretrained_model,
#    Flatten(),
#    Dense(512, activation='relu'),
#    Dense(2, activation='softmax')
# ])


In [7]:
resnet_model = Sequential()

# for layer in pretrained_model.layers:
#         layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(64, activation='relu'))
resnet_model.add(Dense(5, activation='softmax'))

In [8]:
# resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
resnet_model.compile(optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

In [9]:
# Convert to NumPy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

epochs=5
# Train the model
resnet_model.fit(x_train, y_train, epochs=5)
# history = resnet_model.fit(
#     x_train,            # Training data
#     y_train,            # Training labels
#     epochs=epochs,      # Number of epochs
# )

Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 57s 57s/step - accuracy: 0.3000 - loss: 1.6817
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 1.0000 - loss: 0.0310
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 1.0000 - loss: 7.5130e-04
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 1.0000 - loss: 6.1682e-04
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 1.0000 - loss: 4.7099e-04


In [10]:
resnet_model.evaluate(x_train, y_train)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.8000 - loss: 1.6556


[1.6556284427642822, 0.800000011920929]

In [ ]:
# epochs=10
# history = resnet_model.fit(
#   train_dir,
#   validation_data=validation_dir,
#   epochs=epochs
# )

# # Train the model
# history = resnet_model.fit(
#     train_dir,
#     validation_data=validation_dir,
#     epochs=epochs
# )

In [ ]:
# model = models.resnet18(pretrained=True)

# for param in model.parameters():
#     param.requires_grad = False

# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 2)


# model = models.resnet18(pretrained=True)
# num_ftrs = model.fc.in_features

# # Here the size of each output sample is set to 2.
# # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
# model.fc = nn.Linear(num_ftrs, 2)

# device = torch.device('cpu')
# #device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# criterion = nn.CrossEntropyLoss()
# # Observe that all parameters are being optimized
# optimizer = optim.SGD(model.parameters(), lr=0.001)


# step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
# model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=25)